# Recommandation de Produits Basée sur les Tendances
Ce notebook développe un système de recommandation basé sur le contenu :

1. Récupère les mots-clés de tendance et les détails des produits depuis BigQuery.

2. Utilise TF-IDF pour vectoriser les données textuelles.

3. Calcule la similarité cosinus entre les tendances et les produits.

4. Recommande les produits les plus similaires à une tendance donnée.

# Objectif

Recommander des produits qui sont pertinents par rapport à des mots-clés de tendances actuelles.

# Approche

Il s'agit d'un système de recommandation basé sur le contenu (content-based). L'idée est de trouver les produits dont la description textuelle (nom + catégorie) est la plus similaire aux mots-clés des tendances.

# Étapes clés et choix techniques

## 1. Récupération des données

Les mots-clés des tendances (*trend_id, keyword*) et les informations des produits (*product_id, product_name, category*) sont récupérés depuis des tables BigQuery (*trend_data, new_products*).

**Choix :** BigQuery est utilisé comme source de données centralisée et scalable.

## 2. Préparation du texte

Pour chaque produit, le nom (*product_name*) et la catégorie (*category*) sont combinés en un seul champ texte (*product_text*). Les catégories manquantes sont gérées.

**Choix :** Utiliser à la fois le nom et la catégorie enrichit la description textuelle du produit pour une meilleure comparaison sémantique avec les mots-clés des tendances.

## 3. Vectorisation avec TF-IDF

Le texte des produits (*product_text*) et les mots-clés des tendances (*keyword*) sont transformés en vecteurs numériques à l'aide de TF-IDF (Term Frequency-Inverse Document Frequency).

**Choix :**
- TF-IDF est une technique standard et efficace pour représenter l'importance des mots dans un corpus.
- Elle pondère les mots en fonction de leur fréquence dans un document et de leur rareté dans l'ensemble du corpus, mettant en évidence les termes discriminants.
- Les mots courants anglais (*stop words*) sont exclus pour réduire le bruit.
- Le vectoriseur est entraîné sur les produits puis utilisé pour transformer les tendances, assurant un espace vectoriel cohérent.

## 4. Calcul de Similarité

La similarité cosinus est calculée entre chaque vecteur de tendance et chaque vecteur de produit.

**Choix :**
- La similarité cosinus mesure l'angle entre les vecteurs, ce qui est une métrique robuste pour évaluer la similarité sémantique entre des textes représentés vectoriellement (comme ceux issus de TF-IDF).
- Elle est indépendante de la longueur des documents et de la fréquence absolue des termes.

## 5. Génération des Recommandations

- Pour une tendance donnée (*identifiée par trend_id*), le système récupère la ligne correspondante dans la matrice de similarité.
- Les indices des produits ayant les scores de similarité cosinus les plus élevés avec cette tendance sont identifiés.
- Les *top_n* produits les plus similaires (*avec un score > 0*) sont retournés, avec leur identifiant, texte combiné et score de similarité.

# Résumé

Le modèle identifie les produits dont le contenu textuel (*nom + catégorie*) correspond le mieux aux mots-clés d'une tendance donnée, en utilisant **TF-IDF** pour la représentation textuelle et la **similarité cosinus** pour mesurer la pertinence. C'est une approche **classique et interprétable** pour la recommandation basée sur le contenu.




In [15]:
import pandas as pd
from google.cloud import bigquery
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from dotenv import load_dotenv  
import os



In [16]:
# --- Configuration ---
load_dotenv(".envrc")
PROJECT_ID = os.getenv("PROJECT_ID")
DATASET_NAME = os.getenv("DATASET_NAME")
GCS_BUCKET = os.getenv("GCS_BUCKET")
TREND_TABLE = f"{PROJECT_ID}.{DATASET_NAME}.trend_data"
PRODUCT_TABLE = f"{PROJECT_ID}.{DATASET_NAME}.new_products"

# Initialize BigQuery client
# Make sure you are authenticated (e.g., using gcloud auth application-default login)
client = bigquery.Client(project=PROJECT_ID)


## 1. Fetch Data from BigQuery

In [17]:
def fetch_data(query):
    """Executes a BigQuery query and returns a pandas DataFrame."""
    try:
        print(f"Executing query:\n{query}\n")
        query_job = client.query(query)
        results = query_job.result()  # Waits for the job to complete
        df = results.to_dataframe()
        print(f"Fetched {len(df)} rows.")
        return df
    except Exception as e:
        print(f"Error fetching data: {e}")
        return pd.DataFrame() # Return empty DataFrame on error

In [18]:
# Fetch Trends Data (Keywords)
trends_query = f"""
    SELECT DISTINCT
        trend_id,
        keyword
    FROM `{TREND_TABLE}`
    WHERE keyword IS NOT NULL AND TRIM(keyword) != ''
"""
trends_df = fetch_data(trends_query)
trends_df.head()

Executing query:

    SELECT DISTINCT
        trend_id,
        keyword
    FROM `None.None.trend_data`
    WHERE keyword IS NOT NULL AND TRIM(keyword) != ''


Error fetching data: 400 Invalid project ID 'None'. Project IDs must contain 6-63 lowercase letters, digits, or dashes. Some project IDs also include domain name separated by a colon. IDs must start with a letter and may not end with a dash.; reason: invalid, location: None.None.trend_data, message: Invalid project ID 'None'. Project IDs must contain 6-63 lowercase letters, digits, or dashes. Some project IDs also include domain name separated by a colon. IDs must start with a letter and may not end with a dash.

Location: US
Job ID: 7c26c7cc-029d-45d5-8943-2369958c3b86



""


In [5]:
# Fetch Products Data (Name and Category)
products_query = f"""
    SELECT DISTINCT
        product_id,
        product_name,
        category
    FROM `{PRODUCT_TABLE}`
    WHERE product_name IS NOT NULL AND TRIM(product_name) != ''
"""
products_df = fetch_data(products_query)

# Combine product name and category into a single text field for vectorization
# Handle potential missing categories gracefully
products_df['product_text'] = products_df['product_name'] + ' ' + products_df['category'].fillna('')
products_df = products_df[['product_id', 'product_text']].dropna(subset=['product_text'])
products_df.head()

Executing query:

    SELECT DISTINCT
        product_id,
        product_name,
        category
    FROM `trendflow-455409.trendflow.new_products`
    WHERE product_name IS NOT NULL AND TRIM(product_name) != ''




/home/stephane.ristic/.pyenv/versions/3.10.12/envs/TrendFlow/lib/python3.10/site-packages/google/cloud/bigquery/table.py:1933: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


Fetched 10200 rows.


,product_id,product_text
0,1143393e-26c7-41c6-9df9-0ed70ee96f11,So Beauty
1,22485cb8-6c5b-43cd-b88b-5fc9f96d8580,Let Beauty
2,2ba91c1e-6eb7-477c-bac7-5531040f63fd,Another Beauty
3,345a9399-3e44-4da8-bea5-0d72073a03ab,Hour Beauty
4,35e1fbd7-cca7-4ea6-8939-13cfcd56118e,Trade Beauty


## 2. Vectorize Text Data using TF-IDF

In [6]:
# Initialize TF-IDF Vectorizer
# Using stop_words='english' to remove common English words
vectorizer = TfidfVectorizer(stop_words='english', lowercase=True)

# Fit and transform the product descriptions
product_vectors = vectorizer.fit_transform(products_df['product_text'])

# Transform the trend keywords using the same vectorizer
# Note: We only transform trends, don't fit again, to use the product vocabulary
trend_vectors = vectorizer.transform(trends_df['keyword'])

print(f"Product Vectors Shape: {product_vectors.shape}")
print(f"Trend Vectors Shape: {trend_vectors.shape}")

Product Vectors Shape: (10200, 767)
Trend Vectors Shape: (101000, 767)


## 3. Calculate Cosine Similarity

In [7]:
# Calculate cosine similarity between all trends and all products
# Resulting matrix shape: (n_trends, n_products)
similarity_matrix = cosine_similarity(trend_vectors, product_vectors)

print(f"Similarity Matrix Shape: {similarity_matrix.shape}")

Similarity Matrix Shape: (101000, 10200)


## 4. Recommendation Function

In [8]:
def recommend_products_for_trend(trend_id, top_n=5):
    """Recommends top N products for a given trend_id."""
    # Find the index of the trend in our trends_df
    trend_indices = trends_df.index[trends_df['trend_id'] == trend_id].tolist()
    
    if not trend_indices:
        # Try matching by keyword if ID not found or multiple IDs exist for keyword
        trend_keyword_rows = trends_df[trends_df['trend_id'] == trend_id]
        if not trend_keyword_rows.empty:
             keyword = trend_keyword_rows['keyword'].iloc[0]
             trend_indices = trends_df.index[trends_df['keyword'] == keyword].tolist()
             if not trend_indices:
                 print(f"Trend ID {trend_id} or its keyword not found.")
                 return pd.DataFrame()
        else:
             print(f"Trend ID {trend_id} not found.")
             return pd.DataFrame()
            
    # Use the first index found (in case of duplicate keywords/ids)
    trend_index = trend_indices[0]
    trend_keyword = trends_df.loc[trend_index, 'keyword']
    print(f"Finding recommendations for Trend ID: {trend_id} (Keyword: '{trend_keyword}')")
    
    # Get the similarity scores for this trend against all products
    similarity_scores = similarity_matrix[trend_index, :]
    
    # Get the indices of the top N products
    # Use argsort to get indices, then reverse and take top N
    top_product_indices = np.argsort(similarity_scores)[::-1][:top_n]
    
    # Get the corresponding product IDs and scores
    recommended_products = products_df.iloc[top_product_indices].copy()
    recommended_products['similarity_score'] = similarity_scores[top_product_indices]
    
    # Filter out products with zero similarity
    recommended_products = recommended_products[recommended_products['similarity_score'] > 0]
    
    return recommended_products[['product_id', 'product_text', 'similarity_score']]

## 5. Example Usage

In [9]:
# Example: Get recommendations for the first trend_id in the dataframe
if not trends_df.empty:
    example_trend_id = trends_df['trend_id'].iloc[0]
    recommendations = recommend_products_for_trend(example_trend_id, top_n=10)
    print("\nRecommendations:")
    print(recommendations)
else:
    print("No trends data fetched, cannot run example.")

Finding recommendations for Trend ID: 9b21dea8-d653-44df-8111-ccbf012dcd41 (Keyword: 'Mrs')

Recommendations:
                                product_id       product_text  \
3180  4508e208-dab5-4b7f-9684-92e5adb3c3a6       Mrs Clothing   
9252  d53c5d9f-5639-4bcc-9157-35c65c63144d           Mrs Toys   
9251  a7aceb63-5234-4077-aacc-260d4931e66d           Mrs Toys   
1118  788868b0-b615-4d25-8523-d095f8520657         Mrs Beauty   
1117  0f4a5776-8196-4055-92d8-be6a18b21a08         Mrs Beauty   
5245  94bdcc6d-3266-474b-a9ce-974f0e32da16    Mrs Electronics   
5243  1e8035f5-5175-41e6-bc41-f09ce0f22958    Mrs Electronics   
5244  6c352a5e-065b-4e3c-86b3-07ee76c12961    Mrs Electronics   
7236  8f87e7a2-6c9e-49df-95b2-ec608b027609  Mrs Home & Garden   
7235  8c50171c-d4bc-4b29-ae47-61f4417993bb  Mrs Home & Garden   

      similarity_score  
3180          0.947699  
9252          0.946996  
9251          0.946996  
1118          0.946850  
1117          0.946850  
5245          0.946125  